# NER

In [1]:
import numpy as np

## Managing Dataset & Tokenizer

In [2]:
from datasets import load_dataset

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = load_dataset("conll2003", trust_remote_code=True)

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
data["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [6]:
data["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [7]:
data["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [8]:
data["train"].features["ner_tags"].feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [9]:
# Salviamo per dopo
label_names = data["train"].features["ner_tags"].feature.names

In [10]:
from transformers import AutoTokenizer

In [11]:
# Usiamo Distilbert perchè si traina prima.
# Usiamo il cased perchè abbiamo bisogno di distignuere i nomi e può aiutare
checkpoint = "distilbert-base-cased"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
"""
La Named Entity Recognition usa il tokenizer in maniera differente. Al contrario del tokenizer classico che divide le parole
in sottoparole, quando facciamo NER abbiamo bisogno di tenere le parole integre, altrimenti diventa difficile assegnar loro una label correttamente.
Il dataset è già stato splittato in parole, per questo motivo, usiamo "is_split_into_words=True". 
ATTENZIONE: is_split_into_words non serve a creare i token basati su singole parole!!!!
"""
t = tokenizer(data["train"][0]["tokens"], is_split_into_words=True)
t

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
# Il print di t sembra un dizionario. In realtà non lo è. E' un oggetto di tipo BatchEncoding
type(t)

transformers.tokenization_utils_base.BatchEncoding

In [15]:
# Vediamone gli attributi
dir(t)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_encodings',
 '_n_sequences',
 'char_to_token',
 'char_to_word',
 'clear',
 'convert_to_tensors',
 'copy',
 'data',
 'encodings',
 'fromkeys',
 'get',
 'is_fast',
 'items',
 'keys',
 'n_sequences',
 'pop',
 'popitem',
 'sequence_ids',
 'setdefault',
 'to',
 'token_to_chars',
 'token_to_sequence',
 'token_to_word',
 'tok

In [16]:
# Vediamo i token generati:
# Notiamo che la parola "lamb" è stata divisa in due sottoparole.
t.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

## Aligning Tokens to Targets

Come possiamo fare per allineare i token ai targets? Abbiamo due casi specifici da affrontare per adesso:

1) I token speciali come [CLS] e [SEP].
2) Le parole spezzate come "la" e "##mb".

Nel caso 1), ci servirà semplicmente utilizzare un valore da documentazione, -100, che servirà per ignorare totalmente i token. Questo avviene perchè la loss function creata per trainare il transformer è fatta appositamente per avere un gradiente pari a 0 in corrispondenza di -100.

Nel caso 2), si procede ad ESPANDERE il target precedente: semplicemente basta copiare il valore precedente.

In [17]:
sample = data["train"][100]
tokens = sample["tokens"]
targets = sample["ner_tags"]
t = tokenizer(tokens, is_split_into_words=True)
print(t.tokens())
# I word ids sono molto utili. Si ripetono se la parola è spezzata e sono None per i token speciali
print(t.word_ids())

['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']
[None, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]


In [18]:
print(targets)

[1, 0, 0, 0, 0, 0, 0, 0, 0]


In [19]:
# PRIMO TEST, SENZA INPUT
def my_target_aligner(tokens, orig_targets):

    target_cursor = 0
    aligned_targets = []

    for token in tokens:
        if token.startswith("["):
            aligned_targets.append(-100)
        elif token.startswith("##"):
            aligned_targets.append(aligned_targets[-1])
        else:
            aligned_targets.append(orig_targets[target_cursor])
            target_cursor+=1

    return aligned_targets

In [20]:
print(my_target_aligner(t.tokens(), targets))

[-100, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [21]:
# SECONDO TEST CON INPUT DEL WORD ID E DELLA LABEL CONCATENATA
# Begin2Inside necessario perchè a volte possono capitare già ID pari che si susseguono
begin2inside = {
  1: 2,
  3: 4,
  5: 6,
  7: 8,
}

def target_aligner_with_wordids(ids, targets):

    aligned_targets = []
    last_word = None
    
    for wid in ids:
        if wid is None:
            # Qui abbiamo un token speciale da ignorare (id: None)
            label = -100
        elif last_word != wid:
            label = targets[wid]
        else:
            label = targets[wid]
            if label in begin2inside:
                label = begin2inside[label]
        aligned_targets.append(label)
        last_word = wid
            
    return aligned_targets

In [22]:
label_ids = target_aligner_with_wordids(t.word_ids(), targets)
print(label_ids)

[-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [23]:
aligned_labels = [label_names[x] if x>=0 else None for x in label_ids]
aligned_labels

[None, 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', None]

In [24]:
for x,y in zip(t.tokens(), aligned_labels):
    print(f"{x}: {y}")

[CLS]: None
Ra: B-PER
##bino: I-PER
##vich: I-PER
is: O
winding: O
up: O
his: O
term: O
as: O
ambassador: O
.: O
[SEP]: None


In [25]:
def tokenize_fn(batch):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True)
    labels_batch = batch["ner_tags"] # Original targets
    aligned_batch_labels = []
    for i, labels in enumerate(labels_batch):
        word_id = tokenized_inputs.word_ids(i)
        aligned_labels = target_aligner_with_wordids(word_id, labels)
        aligned_batch_labels.append(aligned_labels)

    # Ricordiamo: il nostro target DEVE essere salvato in una colonna chiamata "labels"
    tokenized_inputs["labels"] = aligned_batch_labels

    return tokenized_inputs

In [26]:
# Poi puliamo il risultato eliminando queste colonne
data["train"].column_names

['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']

In [27]:
tokenized_datasets = data.map(tokenize_fn, batched=True, remove_columns=data["train"].column_names)

Map: 100%|███████████████████████████████████████████████████████████████| 3250/3250 [00:00<00:00, 11193.71 examples/s]


In [28]:
tokenized_datasets["train"][:3]

{'input_ids': [[101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  [101, 1943, 14428, 102],
  [101, 26660, 13329, 12649, 15928, 1820, 118, 4775, 118, 1659, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100],
  [-100, 1, 2, -100],
  [-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]]}

## Data Collator

I modelli sono equipaggiati di default con un Data Collator. Questo si occupa di rendere il dato uniforme, creando il giusto padding
e trasformando gli input in tensori automaticamente. Questo viene fatto in automatico dal trainer.

Per i task di "Token Classification" come NER e POS, il data collator deve essere creato a mano ed inserito. Non è niente di eccezionale, la scrittura è spesso ridondante (è necessario inserire il tokenizer che viene poi effettivamente re-inserito nel trainer).

Il collator effettua le operazioni sui batch. Possiamo vedere un esempio dopo

In [29]:
from transformers import DataCollatorForTokenClassification

In [30]:
data_collator =  DataCollatorForTokenClassification(tokenizer=tokenizer)

In [31]:
# Se usiamo questa modalitò, otteniamo un dizionario che comprende liste di liste. Il collator non le accetta
tokenized_datasets["train"][0:2]

{'input_ids': [[101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  [101, 1943, 14428, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]],
 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}

In [32]:
# Per ottenere invece la lista di dizionari, dobbiamo esplicitamente ciclare nel dizionario con list comprehension
[tokenized_datasets["train"][i] for i in range(2)]

[{'input_ids': [101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]},
 {'input_ids': [101, 1943, 14428, 102],
  'attention_mask': [1, 1, 1, 1],
  'labels': [-100, 1, 2, -100]}]

In [33]:
# Notiamo effettivamente come il padding viene applicato col -100
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

## Metrics 

Per quanto riguarda le metriche per valutare il training, ci affidiamo alla metrica "seqeval". Questa metrica si occupa di calcolare effettivamente il valore di precisione delle predizioni, tenendo in conto che abbiamo delle sostanziali differenze rispetto alla normale text-classification.

In primo luogo, abbiamo dei tag. Questi tag seguono il formato BIO (Begin, Internal, Outer) e, dato che la metrica è specifica, classi al di fuori di queste non vengono considerate. Per questo motivo abbiamo bisogno di risalire al tag preciso per fare i calcoli. Questo ci serve per avere un'accuratezza anche in ciò che prediciamo, cercando di discriminare quali tag vengano effettivamente predetti meglio.

In secondo luogo, abbiamo una struttura differente degli output del modello: prima avevamo un set di label (dimensione BatchSize) confrontato con il set di logits predetti (BatchSize x NClasses). Ora, invece, abbiamo una terza dimensione che si unisce ad entrambi gli output, che è la lunghezza dell'input stesso. Quindi avremo BatchSize x Nwords da comparare a BatchSize x NClasses x NWords.

Inoltre, abbiamo il -100 che è l'output ignorato. Va eliminato per evitare di inquinare le predizioni

In [34]:
from evaluate import load
# !pip install seqeval [necessario]

In [35]:
metric = load("seqeval")

In [36]:
# Test1 per errore
# metric.compute(predictions=[0,0,0], references=[0,1,0])

In [37]:
# Test2 per errore -> si aspetta i tag veri. Si aspetta stringhe.
metric.compute(predictions=[["0","0","0"]], references=[["0","1","0"]])

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` paramet

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.6666666666666666}

In [38]:
# Test3 ok
# NB: siccome la libreria sa quello che fa, trovare un "I-ORG" corretto nelle predictions senza il precedente B-ORG viene considerato errore
metric.compute(predictions=[["O","O","I-ORG", "O"]], references=[["O","B-ORG","I-ORG", "O"]])

{'ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.75}

In [39]:
def compute_metrics(logits_and_labels):
    
    logits, labels = logits_and_labels
    preds = np.argmax(logits, axis=-1)

    # rimuoviamo tutti i -100
    # convertiamo i label id in nomi usando la mappa sopa
    str_labels = [[label_names[t] for t in label if t != -100] for label in labels]

    # Stesso per le predizioni, ma con un qualcosa di diverso: controlliamo sulla label se è -100
    str_preds = [[label_names[p] for p, t in zip(pred, targ) if t != -100] for pred, targ in zip(preds, labels)]

    # Sia str_preds che str_labels ora sono liste di liste che hanno al loro interno i tag del NER
    the_metrics = metric.compute(predictions=str_preds, references=str_labels)

    return {
        'precision': the_metrics['overall_precision'],
        'recall': the_metrics['overall_recall'],
        'f1': the_metrics['overall_f1'],
        'accuracy': the_metrics['overall_accuracy'],
    }

    

## Model and Trainer

In [40]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments

In [41]:
# Metodo alternativo per iniettare le labels
id2label = {k:v for k, v in enumerate(label_names)}
label2id = {v:k for k, v in id2label.items()}

In [42]:
model = AutoModelForTokenClassification.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
training_args = TrainingArguments(
    output_dir="distilbert-finetuned-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01 #L2 Regularization
)

In [44]:
# Esempio dato con ner tag 8 esistente
print(data["train"][8])

{'id': '8', 'tokens': ['Fischler', 'proposed', 'EU-wide', 'measures', 'after', 'reports', 'from', 'Britain', 'and', 'France', 'that', 'under', 'laboratory', 'conditions', 'sheep', 'could', 'contract', 'Bovine', 'Spongiform', 'Encephalopathy', '(', 'BSE', ')', '--', 'mad', 'cow', 'disease', '.'], 'pos_tags': [17, 40, 22, 42, 15, 24, 15, 22, 10, 22, 43, 15, 21, 24, 21, 20, 37, 22, 22, 22, 4, 22, 5, 8, 16, 21, 21, 7], 'chunk_tags': [11, 12, 12, 21, 13, 11, 13, 11, 12, 12, 11, 13, 11, 11, 12, 21, 22, 11, 12, 12, 0, 11, 0, 0, 11, 12, 12, 0], 'ner_tags': [1, 0, 7, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0]}


In [45]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [46]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.091100,0.085498,0.879221,0.911478,0.895059,0.976247
2,0.044800,0.074190,0.908462,0.928644,0.918442,0.981751
3,0.027800,0.071450,0.915748,0.936553,0.926034,0.983105


TrainOutput(global_step=5268, training_loss=0.07935355708919517, metrics={'train_runtime': 377.5629, 'train_samples_per_second': 111.566, 'train_steps_per_second': 13.953, 'total_flos': 460431563935266.0, 'train_loss': 0.07935355708919517, 'epoch': 3.0})

In [48]:
from transformers import pipeline

In [49]:
ner = pipeline("token-classification", model="distilbert-finetuned-ner/checkpoint-5268/", aggregation_strategy="simple", 
               device=0)

In [56]:
s = "Bill Gates was the CEO of Microsoft in Seattle, Washington"
ner(s)

[{'entity_group': 'PER',
  'score': 0.99927205,
  'word': 'Bill Gates',
  'start': 0,
  'end': 10},
 {'entity_group': 'ORG',
  'score': 0.9992894,
  'word': 'Microsoft',
  'start': 26,
  'end': 35},
 {'entity_group': 'LOC',
  'score': 0.99927443,
  'word': 'Seattle',
  'start': 39,
  'end': 46},
 {'entity_group': 'LOC',
  'score': 0.9989988,
  'word': 'Washington',
  'start': 48,
  'end': 58}]